# Chat stream

In [1]:
from ollama import chat
from rich import print
from IPython.display import Markdown, display

In [2]:
from config import MODEL, HISTORY_FILE

In [3]:
from tools import get_current_weather, TOOL_SCHEMA, make_friendly

In [4]:
history_file = HISTORY_FILE
history_items = 10
model = MODEL
debug = True
tool_output = True

In [5]:
with open("./prompts/system.md") as file:
    system_prompt = file.read()

In [6]:
import json

def write_messages(messages, messages_file: str = HISTORY_FILE):
    with open(messages_file, "r") as f:
        data = json.load(f)

    data.extend(messages)

    json_data = json.dumps(data)

    with open(messages_file, "w") as f:
        f.write(json_data)

In [7]:
# while True:
#   user_input = input('You: ')
#   if user_input == "/quit":
#       raise SystemExit("Stop here")
#   response = chat(
#     model=model,
#     messages=messages
#     + [
#       {'role': 'user', 'content': user_input},
#     ],
#   )

#   # Add the response to the messages to maintain the history
#   newest_messages = [
#     {'role': 'user', 'content': user_input},
#     {'role': 'assistant', 'content': response.message.content},
#   ]
#   messages += newest_messages

    
#   print(response.message.content + '\n')
#   write_messages(newest_messages)

In [9]:
# update chat memory from history
with open(history_file) as file:
    history = json.load(file)[:history_items]

messages = [
  {
      'role': 'system',
      'content': system_prompt,
  },
]

messages.extend(history)

display(Markdown("#### 🧒 You:"))
# start chat
while True:
    user_input = input('')

    if user_input == "/quit":
        raise SystemExit("Stop here")
    else:
        display(Markdown("_Thinking..._"))
    response = chat(
        model=model,
        tools=TOOL_SCHEMA,
        messages=messages
        + [
            {'role': 'user', 'content': user_input},
        ],
    )

    ### new logic for calling tools
    if getattr(response.message, 'tool_calls', None):
        if debug:
            print(f"Found {len(response.message.tool_calls)} tool calls")
            print(response.message.tool_calls)
        func_name = response.message.tool_calls[0].function.name # test on one func for now
        if func_name in globals():
            func = globals()[func_name]
            params = response.message.tool_calls[0].function.arguments
            if debug:
                print(f"Calling function {func_name}")
                print(params)
            func_output = func(params)
            if tool_output:
                display(Markdown(f"---\n\n#### 🔧 Tool `{func_name}`\n\n{func_output}\n\n---"))

            # make friendly
            if debug:
                print("Making friendly")
            response = make_friendly(func_output)
            
            
        else:
            print("Can't find function")
    else:
        print("No tool calls found")
    ### end tool logic

    # Add the response to the messages to maintain the history
    newest_messages = [
        {'role': 'user', 'content': user_input},
        {'role': 'assistant', 'content': response.message.content},
    ]
    messages += newest_messages

    print(response)
    display(Markdown(f"---\n\n#### 🤖 Assistant:\n\n{response.message.content} \n\n---\n\n#### 🧒 You:"))
    
    write_messages(newest_messages)

#### 🧒 You:

 how's the weather in berlin?


_Thinking..._

Found 1 tool calls

[ToolCall(function=Function(name='get_current_weather', arguments={'city': 'Berlin'}))]

Calling function get_current_weather

{'city': 'Berlin'}

{'city': 'Berlin'}
http://wttr.in/berlin?format="Current+weather+for+%l:+%C,+Temperature:+%t,+Feels+like:+%f,+Precipitation+(mm, next+3+hours):+%p,+Wind:+%w"


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))